In [1]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from collections import defaultdict
import random
import csv

In [2]:
# Define the parameters

# Path to the CSV file containing the cost data
data_path = "./cost_10_07.csv"

# Column name in the CSV file that represents the part
part_col = "Part"

# Column name in the CSV file that represents the departure/arrival site
dep_arr_site_col = "Departure/Arrival site"

# Column name in the CSV file that represents the arrival/departure site
arr_dep_site_col =  "Arrival/Departure site"

# Column name in the CSV file that represents the cost
cost_col = "Cost"

# List of tuples representing the part and its corresponding site
# Each tuple is in the format (part, site)
PBS = [(2, 1), (3, 1), (4, 1), (5, 2) , (6, 2), (7, 2), (8, 3), (9, 4), (10, 4)]

# List of tuples representing the subparts
# Each tuple is in the format (subpart1, subpart2)
subpart = [(2,3),(2,4),(3,4),(5,6),(5,7),(6,7),(9,10)]

# Penalty parameters for the optimization problem
lambda_1 = 10
lambda_2 = 10
lambda_3 = 10

In [3]:
# Read data from the CSV file
data = pd.read_csv(data_path)
# Store it in a pandas dataframe
data_df = pd.DataFrame(data)

In [4]:
# Define the function to create the cost matrix DataFrame
def cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col):

    # Define the range of parts and sites
    # The range of parts starts from 2 and goes up to the maximum part number
    # The range of sites starts from 1 and goes up to the maximum site number
    part_range = range(2, data_df[part_col].max() + 1)
    site_range = range(1, max(data_df[dep_arr_site_col].max(), data_df[arr_dep_site_col].max()) + 1)

    pivot_df = (
        pd.DataFrame(
            index=pd.MultiIndex.from_product(
                [part_range, site_range, site_range], 
                names=['Part', 'Departure_Site', 'Arrival_Site']
            ) # Create a pivot DataFrame with a multi-index of parts, departure sites, and arrival sites
        )
        .reset_index() # Rest index of the dataframe
        .merge(data_df.assign(Part_Column=data_df[part_col]), how='left'
               , left_on=['Part', 'Departure_Site', 'Arrival_Site'] # Merge the pivot DataFrame with the original DataFrame on the part, departure site, and arrival site columns
               , right_on=[part_col, dep_arr_site_col, arr_dep_site_col])
        .assign(Cost=lambda df: df[cost_col].fillna(0)) # Fill the missing cost values with 0
        .pivot_table(index=['Part', 'Departure_Site']
                     , columns=['Part_Column', 'Arrival_Site']
                     , values='Cost'
                     , fill_value=0) # Pivot the DataFrame to create a cost matrix with the part and departure site as the index and the part and arrival site as the columns
        .pipe(lambda df: df.reorder_levels(['Part_Column', 'Arrival_Site'], axis=1))    # Reorder the levels of the columns
        .pipe(lambda df: df.reindex(columns=pd.MultiIndex.from_product([part_range, site_range]
                                                                       , names=df.columns.names)
                                    , fill_value=0)) # Reindex the columns and the index to include all combinations of parts and sites
        .pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([part_range, site_range], names=df.index.names), fill_value=0))
        .pipe(lambda df: df + df.T) # Make the DataFrame symmetric by adding its transpose to itself
        .pipe(lambda df: df.where(np.eye(df.shape[0], dtype=bool), df / 2)) # Halve the off-diagonal elements and keep the diagonal the same
    )

    # Return the pivot DataFrame
    return pivot_df

In [5]:
# Define the function to create the cost function
def cost_function_qubo(PBS):
    # Get the range of parts and sites by calling the cost_matrix_df function
    # This function returns a pivot DataFrame with the cost matrix
    pivot_df = cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)
    
    # Get the unique values of 'Departure_Site' from the pivot DataFrame's index
    site_range = pivot_df.index.get_level_values('Departure_Site').unique()

    # Convert the pivot DataFrame to a QUBO dictionary
    # The QUBO dictionary is created by iterating over the PBS list and the site range
    # For each pair of sites and each pair of parts, if the sites are different and the parts are different,
    # and if the pair of parts and the first site and the pair of parts and the second site are in the pivot DataFrame's index,
    # then add an entry to the QUBO dictionary with the key as the pair of parts and sites and the value as the corresponding cost from the pivot DataFrame
    cost_function = {((r, i), (s, j)): pivot_df.loc[(r, i), (r, j)] 
            for r, s in PBS for i in site_range for j in site_range if i != j and r != s and (r, i) in pivot_df.index and (r, j) in pivot_df.index}

    # Return the QUBO dictionary with the cost function
    return cost_function

In [6]:
# Define the function to create the first constraint function
def penalty_C1_qubo(PBS):
    
    # Determine the maximum number of sites from the 'Arrival/Departure site' column of the DataFrame
    max_sites = data_df['Arrival/Departure site'].max()

    # Define default penalty
    penalty = 1
    
    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C1 = {}
    
    # Iterate over the PBS list
    for r, _ in PBS:
        # Iterate over the range from 1 to the maximum number of sites
        for i in range(1, max_sites + 1):
            # Add a penalty term to the QUBO dictionary for each site
            penalty_qubo_C1[((r, i), (r, i))] = - penalty
            # Iterate over the range from the current site number + 1 to the maximum number of sites
            for j in range(i + 1, max_sites + 1):
                # Add penalty terms to the QUBO dictionary for each pair of sites
                penalty_qubo_C1[((r, i), (r, j))] = 2 * penalty
                penalty_qubo_C1[((r, j), (r, i))] = 2 * penalty

    # Return the QUBO dictionary with the penalty terms
    return penalty_qubo_C1

In [7]:
# Define the function to create the second constraint function
def penalty_C2_qubo(PBS):
    
    # Determine the maximum number of sites from the 'Arrival/Departure site' column of the DataFrame
    max_sites = data_df['Arrival/Departure site'].max()

    # Define default penalty
    penalty = 1
    
    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C2 = {}
    
    # Iterate over the PBS list
    for r, s in PBS:
        # Iterate over the range from 1 to the maximum number of sites
        for i in range(1, max_sites + 1):
            # Add a penalty term to the QUBO dictionary for each site
            # The penalty is added if the product of Xr,i and Xs,i is non-zero
            penalty_qubo_C2[((r, i), (s, i))] = penalty

    # Return the QUBO dictionary with the penalty terms
    return penalty_qubo_C2

In [8]:
# Define the function to create the third constraint function
def penalty_C3_qubo(subpart):
    
    # Determine the maximum number of sites from the 'Arrival/Departure site' column of the DataFrame
    max_sites = data_df['Arrival/Departure site'].max()

    # Create a QUBO dictionary to hold the penalty terms
    penalty_qubo_C3 = {}
    
    # Define default penalty
    penalty = 1
    
    # Iterate over the subpart list
    for r, s in subpart:
        # Check if r is less than s
        if r < s:
            # Iterate over the range from 1 to the maximum number of sites
            for i in range(1, max_sites + 1):
                # Add a penalty term to the QUBO dictionary for each site
                # The penalty is added if the origins of two subparts of a common part are the same
                penalty_qubo_C3[((r, i), (s, i))] = penalty

    # Return the QUBO dictionary with the penalty terms
    return penalty_qubo_C3

In [9]:
# Auxiliary function to multiply penalties by QUBO dictionaries

def multiply_penalty_qubo(penalty, qubo_dict):
    # Create a new QUBO dictionary to hold the result
    result_qubo = {}

    # Multiply each value in the QUBO dictionary by the penalty
    for key, value in qubo_dict.items():
        result_qubo[key] = penalty * value

    return result_qubo

In [10]:
# Auxiliary function to add QUBO dictionaries
def add_qubo_dicts(qubo_dict1, qubo_dict2):
    # Create a new defaultdict to hold the result
    result_qubo = defaultdict(int)

    # Add the values from the first QUBO dictionary
    for key, value in qubo_dict1.items():
        result_qubo[key] += value

    # Add the values from the second QUBO dictionary
    for key, value in qubo_dict2.items():
        result_qubo[key] += value

    return dict(result_qubo)

In [11]:
def calculate_qubo(penalties_qubos):
    # Initialize Q as an empty dictionary
    Q = {}
    # Iterate over the list of tuples
    for penalty, qubo in penalties_qubos:
        # Multiply the QUBO dictionary by its penalty
        qubo_multiplied = multiply_penalty_qubo(penalty, qubo)
        
        # Add the multiplied QUBO dictionary to Q
        Q = add_qubo_dicts(Q, qubo_multiplied)
    
    return Q

In [12]:
def qubo_matrix_df(Q):
    Q_df = (
        pd.Series(Q)  # Convert the QUBO dictionary into a pandas Series
        .pipe(lambda s: pd.DataFrame(s, index=pd.MultiIndex.from_tuples(s.index)))  # Create a DataFrame with a multi-index from the keys of the QUBO dictionary
        .unstack(level=0)  # Unstack the DataFrame to create a matrix
        .transpose()  # Transpose the DataFrame
        .fillna(0)  # Fill NaN values with 0
        .pipe(lambda df: df if df.index.name is None else df.rename_axis(None))  # Remove the index name
        .pipe(lambda df: df.set_index(df.index.map(lambda x: str(x[1]) if isinstance(x, tuple) else str(x))))  # Convert the multi-index into a string representation of the tuple
    )

    return Q_df

In [13]:
# C is the QUBO matrix for the cost function.
# The cost_function_qubo function takes PBS as an argument and returns
# the QUBO matrix.
C = cost_function_qubo(PBS)

# C1 is the QUBO matrix for the first penalty term.
# The penalty_C1_qubo function takes PBS as an argument and returns
# the QUBO matrix for the first penalty term.
C1 = penalty_C1_qubo(PBS)

# C2 is the QUBO matrix for the second penalty term.
# The penalty_C2_qubo function takes PBS as an argument and returns
# the QUBO matrix for the second penalty term.
C2 = penalty_C2_qubo(PBS)

# C3 is the QUBO matrix for the third penalty term.
# The penalty_C3_qubo function takes subpart as an argument and returns
# the QUBO matrix for the third penalty term.
C3 = penalty_C3_qubo(subpart)

In [14]:
# Introduce the structure of the Qubo problem
penalties_qubos = [(1, C), (lambda_1, C1), (lambda_2, C2), (lambda_3, C3)]
# Calculate the Qubo matrix
Q = calculate_qubo(penalties_qubos)
# QUBO matrix as a df for inspection
Q_df = qubo_matrix_df(Q)
# Show Q_df
Q_df

,"(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)","(1, 6)","(1, 7)","(2, 1)","(2, 2)","(2, 3)",...,"(9, 5)","(9, 6)","(9, 7)","(10, 1)","(10, 2)","(10, 3)","(10, 4)","(10, 5)","(10, 6)","(10, 7)"
"(2, 1)",10.000,0.820,0.525,0.545,0.715,0.455,0.850,-10.0,20.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 2)",0.820,10.000,0.295,0.185,0.560,0.120,0.895,20.0,-10.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 3)",0.525,0.295,10.000,0.465,0.510,0.675,0.825,20.0,20.0,-10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 4)",0.545,0.185,0.465,10.000,0.825,0.095,0.760,20.0,20.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(2, 5)",0.715,0.560,0.510,0.825,10.000,0.520,0.350,20.0,20.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(10, 3)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,20.0,-10.0,20.0,20.0,20.0,20.0
"(10, 4)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,20.0,20.0,-10.0,20.0,20.0,20.0
"(10, 5)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,20.0,20.0,20.0,-10.0,20.0,20.0
"(10, 6)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,20.0,20.0,20.0,20.0,-10.0,20.0
